# Import

In [18]:
import qiskit as qs
import numpy as np
from qiskit import IBMQ, execute
import qiskit.pulse as pulse
from qiskit.pulse import Play
from qiskit.compiler import assemble
from qiskit.tools.monitor import job_monitor
from scipy.optimize import curve_fit

# Linking to the Ibm quantum processor using qiskit

In [3]:
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [4]:
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')

In [5]:
qubit = 0
qubit_frequency = backend.properties().qubit_property(qubit, "frequency")[0]

In [6]:
qubit_frequency

4971677927.931224

# Scheduling an experiment

In [7]:
pulse_amp_array =np.linspace(0.1, 1, 10)
pulse_times = np.array(
    [4 + np.arange(0, int(36. / (amplitude)), 6) for amplitude in pulse_amp_array]
)
pulse_times = pulse_times * 16

C:\Users\soohw\AppData\Local\Temp\ipykernel_14380\1100997428.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pulse_times = np.array(


In [12]:
backend_defaults = backend.defaults()
inst_sched_map = backend_defaults.instruction_schedule_map
measure_schedule = inst_sched_map.get("measure", qubits=[qubit])
drive_chan = pulse.DriveChannel(qubit)

In [16]:
rabi_programs_dic_I = {}
num_shots_per_point = 1024
for idx, pulse_amplitude in enumerate(pulse_amp_array):
    rabi_schedules_I = []
    for duration_pulse in pulse_times[idx]:
        drive_pulse = qs.pulse.library.gaussian_square(
            duration=duration_pulse,
            sigma=1,
            amp=pulse_amplitude,
            risefall=1,
            name=f"square_pulse_{duration_pulse}",
        )
        schedule = pulse.Schedule(name=str(duration_pulse))
        schedule |= (
            Play(drive_pulse, pulse.DriveChannel(qubit)) << schedule.duration
        )
        schedule += measure_schedule << schedule.duration
        rabi_schedules_I.append(schedule)
    rabi_experiment_program_I = assemble(
        rabi_schedules_I,
        backend=backend,
        meas_level=2,
        meas_return="single",
        shots=num_shots_per_point,
        schedule_los=[{drive_chan: qubit_frequency}]
        * len(pulse_times[idx]),
    )
    rabi_programs_dic_I[pulse_amplitude] = rabi_experiment_program_I

In [22]:
def fit_function(x_val, y_val, func, bound):
    fitparams, conv = curve_fit(func, x_val, y_val, bounds=bound)
    y_fit = func(x_val, *fitparams)
    return fitparams, y_fit
dt = backend.configuration().dt

# Running calibration experiments

In [27]:
rabi_calibration_exp_I = []
rabi_oscillations_results = []
for idx, pulse_amplitude in enumerate(pulse_amp_array):
    job = backend.run(rabi_programs_dic_I[pulse_amplitude])
    print("Runing progress ",idx+1," out of ",len(pulse_amp_array))
    job_monitor(job)
    rabi_results = job.result(timeout=120)
    rabi_values = []
    time_array = pulse_times[idx] * dt
    for time_idx in pulse_times[idx]:
        counts = rabi_results.get_counts(str(time_idx))
        excited_pop = 0
        for bits, count in counts.items():
            excited_pop += count if bits[::-1][qubit] == "1" else 0
        rabi_values.append(excited_pop / num_shots_per_point)

    rabi_oscillations_results.append(rabi_values)
    fit_parameters, y_fit = fit_function(
        time_array,
        rabi_values,
        lambda x, A, rabi_freq, phi: A
        * np.cos(2 * np.pi * rabi_freq * x + phi) ** 2,
        (
            [0.9, np.abs(pulse_amplitude * 0.4 * 1e7), 1],
            [1, np.abs(pulse_amplitude * 1.2 * 1e7), 2],
        ),
    )

    rabi_calibration_exp_I.append(fit_parameters[1])


Runing progress  1  out of  10
Job Status: job is queued (6)     

KeyboardInterrupt: 